# <span style= "color:#57a989">Notebook BoW</span>
**Materia:** Introducción a la Ciencia de Datos

**Autor:** Eber David Gaytán Medina

#### Instalación e importación de paquetes

In [ ]:
#!pip install spacy
#!pip install nltk
#!pip install sklearn

In [1]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

#### Instalar esto en caso de error al usar tokenizador (nltk)
```python
nltk.download('punkt')
nltk.download('punkt_tab')
```

#### Instalar esto en caso de error al usar stopwords (nltk)
```python
nltk.download('stopwords')
```

#### Leer archivo csv

In [2]:
df = pd.read_csv(r'df_mini_HS.csv')

In [3]:
print(df)

    label                                               text
0       1  ESAS COSAS Y OTRAS PUEDEN PASAR POR MANTENER A...
1       1  28: te amodio, odio a la perra de tu amiga ☺️☺...
2       1  @LaDivinaDiva Callate maldita perra. O seguro ...
3       1  @MarysabelPuerto Mejor callate cara de puta o ...
4       1  @xarita327 @TRIKYHUMOR @yonier2012 @casTa1326 ...
5       1              @CocotePR @ashleyhonohan callate puta
6       1  Y el inmigrante recibe ayuda del rico Estado l...
7       1  De los moros no se puede esperar nada bueno, y...
8       1  ¿Por que si a una mujer le pegan un tiro en la...
9       1  Analicemos esto: ¿Si te pones unos shorts así,...
10      0  "See... Tal vez les recordo como Peron protegi...
11      0  PIETRAPIERCE STORY: PURS SANGS ARABES STARS DE...
12      0  ¿Qué dice este de frivolizar el acoso escolar?...
13      0  #Suiza 🇨🇭 retira el permiso a 189 refugiados q...
14      0  Hoy quiero denunciaaaaaaar A LA GENTE PUTO GUA...
15      0  Redomicilie s

#### Normalización de datos y limpieza

In [4]:
df['text'] = df['text'].str.lower() #Cambiar todas las letras a minusculas
df['text'] = df['text'].str.replace(r"@\S+", "", regex=True)# Eliminar menciones a usuarios
df['text'] = df['text'].str.replace(r"http[s]?\://\S+", "", regex=True)# Eliminar enlaces
df['text'] = df['text'].str.replace(r"#\S+", "", regex=True)# Eliminar hashtags
df['text'] = df['text'].str.replace(r"[0-9]", "", regex=True)# Eliminar números
df['text'] = df['text'].str.replace(r"(\(.*\))|(\[.*\])", "", regex=True)# Eliminar paréntesis y corchetes
df['text'] = df['text'].str.replace(r"\n", "", regex=True)# Eliminar caracteres de nueva línea
df['text'] = df['text'].str.replace(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", regex=True)# Eliminar varios patrones en una sola expresión regular (enlaces, paréntesis/corchetes, hashtags, menciones y nueva línea)
df['text'] = df['text'].str.replace(r"(\.)|(,)", "", regex=True)# Eliminar puntos y comas
df['text'] = df['text'].str.replace(r"[¡!]", "", regex=True)# Eliminar signos de admiración
df['text'] = df['text'].str.replace(r"[¿?]", "", regex=True) # Eliminar signos de exclamación


In [5]:
print(df["text"])

0     esas cosas y otras pueden pasar por mantener a...
1     : te amodio odio a la perra de tu amiga ☺️☺️pe...
2      callate maldita perra o seguro eres un pobre ...
3      mejor callate cara de puta o reputa como tu m...
4                                          cállate puta
5                                          callate puta
6     y el inmigrante recibe ayuda del rico estado l...
7     de los moros no se puede esperar nada bueno y ...
8     por que si a una mujer le pegan un tiro en la ...
9     analicemos esto: si te pones unos shorts así e...
10    "see tal vez les recordo como peron protegio a...
11    pietrapierce story: purs sangs arabes stars de...
12    qué dice este de frivolizar el acoso escolar p...
13     🇨🇭 retira el permiso a  refugiados que fueron...
14    hoy quiero denunciaaaaaaar a la gente puto gua...
15    redomicilie su sociedad offshore en emiratos á...
16     basta poned pie en pared a tanta provocación ...
17    semana de la juventud torneo futbol  / fut

#### Separacion de texto por clase. 
El texto de clase "1" y "0" se convierten y se combinan en un solo string

In [6]:
texts_label_1 = df[df["label"] == 1]["text"].astype(str).tolist()
combined_text_label_1 = ' '.join(texts_label_1)

In [7]:
texts_label_0 = df[df["label"] == 0]["text"].astype(str).tolist()
combined_text_label_0 = ' '.join(texts_label_0)

Se pasa un filtro mas para quitar emojis y otros simbolos a cada string. Más adelante se filtrara mediante otro modelo.

In [8]:
combined_text_label_0 = re.sub(r'[\U0001F600-\U0001F64F\u263a-\U0001f645\:\*\-\"\/]', '', combined_text_label_0)

In [9]:
combined_text_label_1 = re.sub(r'[\U0001F600-\U0001F64F\u263a-\U0001f645\:\*\-\"\/]', '', combined_text_label_1)

In [10]:
print(combined_text_label_0,combined_text_label_1)

see tal vez les recordo como peron protegio a eichmann y cientos de nazis o les conto a los arabes el chiste del araba que se lo garcho un camello  pietrapierce story purs sangs arabes stars des festivites a agar el m  qué dice este de frivolizar el acoso escolar por favor queréis dejar de decir semejantes tonterías    retira el permiso a  refugiados que fueron de vacaciones a sus países    vía  hoy quiero denunciaaaaaaar a la gente puto guarra que huele a sudor y chorizo y se sube al bus dejando a tol mundo ko shame on you 🤬 redomicilie su sociedad offshore en emiratos árabes unidos   basta poned pie en pared a tanta provocación y cortad la humillación de estos cuatro hijos de perra semana de la juventud torneo futbol   futbol  categoria cadete equipos inscritos los yogurines la elite rayo donbenitense los negratas fuenlabrada minato de kiev los josewifakers voolka    callate y metete party de una puta vez cuántos inmigrantes creemos que hay y cuántos hay en realidad ciudadanos de un 

#### Tokeniza cada clase

In [11]:
tokens0 = word_tokenize(combined_text_label_0)
print(tokens0)

['see', 'tal', 'vez', 'les', 'recordo', 'como', 'peron', 'protegio', 'a', 'eichmann', 'y', 'cientos', 'de', 'nazis', 'o', 'les', 'conto', 'a', 'los', 'arabes', 'el', 'chiste', 'del', 'araba', 'que', 'se', 'lo', 'garcho', 'un', 'camello', 'pietrapierce', 'story', 'purs', 'sangs', 'arabes', 'stars', 'des', 'festivites', 'a', 'agar', 'el', 'm', 'qué', 'dice', 'este', 'de', 'frivolizar', 'el', 'acoso', 'escolar', 'por', 'favor', 'queréis', 'dejar', 'de', 'decir', 'semejantes', 'tonterías', 'retira', 'el', 'permiso', 'a', 'refugiados', 'que', 'fueron', 'de', 'vacaciones', 'a', 'sus', 'países', 'vía', 'hoy', 'quiero', 'denunciaaaaaaar', 'a', 'la', 'gente', 'puto', 'guarra', 'que', 'huele', 'a', 'sudor', 'y', 'chorizo', 'y', 'se', 'sube', 'al', 'bus', 'dejando', 'a', 'tol', 'mundo', 'ko', 'shame', 'on', 'you', '🤬', 'redomicilie', 'su', 'sociedad', 'offshore', 'en', 'emiratos', 'árabes', 'unidos', 'basta', 'poned', 'pie', 'en', 'pared', 'a', 'tanta', 'provocación', 'y', 'cortad', 'la', 'humill

In [12]:
tokens1 = word_tokenize(combined_text_label_1)
print(tokens1)

['esas', 'cosas', 'y', 'otras', 'pueden', 'pasar', 'por', 'mantener', 'a', 'la', 'inmigracion', 'ilegal', 'en', 'españa', 'y', 'no', 'expulsarlos', 'no', 'la', 'agresión', 'de', 'los', 'manteros', 'a', 'un', 'turista', 'pone', 'a', 'colau', 'en', 'un', 'aprieto', 'vía', 'te', 'amodio', 'odio', 'a', 'la', 'perra', 'de', 'tu', 'amiga', 'pero', 'tú', 'eres', 'mi', 'vida', 'tijerazo', 'callate', 'maldita', 'perra', 'o', 'seguro', 'eres', 'un', 'pobre', 'marico', 'detras', 'de', 'un', 'user', 'femenino', 'chavista', 'colaborador', 'mejor', 'callate', 'cara', 'de', 'puta', 'o', 'reputa', 'como', 'tu', 'madre', 'se', 'nota', 'que', 'te', 'hacen', 'falta', 'estos', 'cállate', 'puta', 'callate', 'puta', 'y', 'el', 'inmigrante', 'recibe', 'ayuda', 'del', 'rico', 'estado', 'ladrón', 'que', 'se', 'olvida', 'de', 'los', 'nacionales', 'mientras', 'nos', 'viola', 'y', 'mata', 'pintada', 'con', 'una', 'verdad', 'a', 'medias', 'de', 'los', 'moros', 'no', 'se', 'puede', 'esperar', 'nada', 'bueno', 'y', 

#### Stopwords

In [13]:
spanish_stopwords = stopwords.words('spanish')
palabras_filtradas0 = [palabra for palabra in tokens0 if palabra not in spanish_stopwords]
print(palabras_filtradas0)

['see', 'tal', 'vez', 'recordo', 'peron', 'protegio', 'eichmann', 'cientos', 'nazis', 'conto', 'arabes', 'chiste', 'araba', 'garcho', 'camello', 'pietrapierce', 'story', 'purs', 'sangs', 'arabes', 'stars', 'des', 'festivites', 'agar', 'm', 'dice', 'frivolizar', 'acoso', 'escolar', 'favor', 'queréis', 'dejar', 'decir', 'semejantes', 'tonterías', 'retira', 'permiso', 'refugiados', 'vacaciones', 'países', 'vía', 'hoy', 'quiero', 'denunciaaaaaaar', 'gente', 'puto', 'guarra', 'huele', 'sudor', 'chorizo', 'sube', 'bus', 'dejando', 'tol', 'mundo', 'ko', 'shame', 'on', 'you', '🤬', 'redomicilie', 'sociedad', 'offshore', 'emiratos', 'árabes', 'unidos', 'basta', 'poned', 'pie', 'pared', 'tanta', 'provocación', 'cortad', 'humillación', 'cuatro', 'hijos', 'perra', 'semana', 'juventud', 'torneo', 'futbol', 'futbol', 'categoria', 'cadete', 'equipos', 'inscritos', 'yogurines', 'elite', 'rayo', 'donbenitense', 'negratas', 'fuenlabrada', 'minato', 'kiev', 'josewifakers', 'voolka', 'callate', 'metete', '

In [14]:
palabras_filtradas1 = [palabra for palabra in tokens1 if palabra not in spanish_stopwords]
print(palabras_filtradas1)

['cosas', 'pueden', 'pasar', 'mantener', 'inmigracion', 'ilegal', 'españa', 'expulsarlos', 'agresión', 'manteros', 'turista', 'pone', 'colau', 'aprieto', 'vía', 'amodio', 'odio', 'perra', 'amiga', 'vida', 'tijerazo', 'callate', 'maldita', 'perra', 'seguro', 'pobre', 'marico', 'detras', 'user', 'femenino', 'chavista', 'colaborador', 'mejor', 'callate', 'cara', 'puta', 'reputa', 'madre', 'nota', 'hacen', 'falta', 'cállate', 'puta', 'callate', 'puta', 'inmigrante', 'recibe', 'ayuda', 'rico', 'ladrón', 'olvida', 'nacionales', 'mientras', 'viola', 'mata', 'pintada', 'verdad', 'medias', 'moros', 'puede', 'esperar', 'bueno', 'dicen', 'propios', 'árabes', 'luchado', 'si', 'mujer', 'pegan', 'tiro', 'cabeza', 'dura', 'tres', 'dias', 'morirse', 'tres', 'dias', 'bala', 'encuentra', 'cerebro', 'analicemos', 'si', 'pones', 'shorts', 'así', 'calle', 'esperas', 'digan', 'acoso', 'provocación']


#### Uso del modelo es_core_news_sm
Descargar nel modelo mediante el siguiente comando:
```bash
!python3 -m spacy download es_core_news_sm
```

In [15]:
nlp = spc.load("es_core_news_sm")
lema = nlp(" ".join(palabras_filtradas0))
oracion_lematizada0 = " ".join([token.lemma_ for token in lema])
print(oracion_lematizada0)

see tal vez recordo peron protegio eichmann ciento nazi conto arab chiste arar garcho camello pietrapierce story purs sang arab stars des festivit agar m decir frivolizar acoso escolar favor queréis dejar decir semejante tontería retirar permiso refugiado vacación país vía hoy quiero denunciaaaaaaar gente puto guarra oler sudor chorizo subir bus dejar tol mundo ko shame on you 🤬 redomicilie sociedad offshore emirato árabe unido bastar poned pie pared tanto provocación cortad humillación cuatro hijo perra semana juventud torneo futbol futbol categoria cadete equipo inscrito yogurín elite rayo donbenitensar negrata fuenlabrado minato kiev josewifakers voolka callate metete party puta vez cuánto inmigrante creer cuánto realidad ciudadano lugar llamado mundo


In [16]:
nlp = spc.load("es_core_news_sm")
lema = nlp(" ".join(palabras_filtradas1))
oracion_lematizada1 = " ".join([token.lemma_ for token in lema])
print(oracion_lematizada1)

cosa poder pasar mantener inmigracion ilegal españa expulsar él agresión mantero turista poner colau aprieto vía amodio odio perra amigo vida tijerazo callate maldita perra seguro pobre marico detras user femenino chavista colaborador mejor callate cara puta reputa madre nota hacer falta cállate puto callate puta inmigrante recibir ayuda rico ladrón olvidar nacional mientras viola mata pintado verdad medio moro poder esperar bueno decir propio árabe luchado si mujer peguir tiro cabeza durar tres dia morir él tres dia bala encontrar cerebro analicer si pón shorts así calle esperas decir acoso provocación


#### Vectorizacion de las lineas de texto

In [17]:
vectorizador0 = CountVectorizer()
vectores0 = vectorizador0.fit_transform([oracion_lematizada0])
vocabulario0 = vectorizador0.get_feature_names_out()

In [18]:
vectorizador1 = CountVectorizer()
vectores1 = vectorizador1.fit_transform([oracion_lematizada1])
vocabulario1 = vectorizador1.get_feature_names_out()

#### Imprimir vector BoW por clase

In [19]:
print("Oración lematizada:", oracion_lematizada1)
print("Vectores Bag of Words:", vectores1.toarray())
print("Vocabulario:", vocabulario1)

Oración lematizada: cosa poder pasar mantener inmigracion ilegal españa expulsar él agresión mantero turista poner colau aprieto vía amodio odio perra amigo vida tijerazo callate maldita perra seguro pobre marico detras user femenino chavista colaborador mejor callate cara puta reputa madre nota hacer falta cállate puto callate puta inmigrante recibir ayuda rico ladrón olvidar nacional mientras viola mata pintado verdad medio moro poder esperar bueno decir propio árabe luchado si mujer peguir tiro cabeza durar tres dia morir él tres dia bala encontrar cerebro analicer si pón shorts así calle esperas decir acoso provocación
Vectores Bag of Words: [[1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 2 1 1 1 1 1 1 1 2 1
  1 2 1 1 1 1 1 1 1 2]]
Vocabulario: ['acoso' 'agresión' 'amigo' 'amodio' 'analicer' 'aprieto' 'así' 'ayuda'
 'bala' 'bueno' 'cabeza' 'callate' 'calle' 'cara' 'cerebro' 'chavista'
 'colaborador' 'co

In [20]:
print("Oración lematizada:", oracion_lematizada0)
print("Vectores Bag of Words:", vectores0.toarray())
print("Vocabulario:", vocabulario0)

Oración lematizada: see tal vez recordo peron protegio eichmann ciento nazi conto arab chiste arar garcho camello pietrapierce story purs sang arab stars des festivit agar m decir frivolizar acoso escolar favor queréis dejar decir semejante tontería retirar permiso refugiado vacación país vía hoy quiero denunciaaaaaaar gente puto guarra oler sudor chorizo subir bus dejar tol mundo ko shame on you 🤬 redomicilie sociedad offshore emirato árabe unido bastar poned pie pared tanto provocación cortad humillación cuatro hijo perra semana juventud torneo futbol futbol categoria cadete equipo inscrito yogurín elite rayo donbenitensar negrata fuenlabrado minato kiev josewifakers voolka callate metete party puta vez cuánto inmigrante creer cuánto realidad ciudadano lugar llamado mundo
Vectores Bag of Words: [[1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

**Se crea el dataframe agregando los vocabularios como encabezados**

In [21]:
df0 = pd.DataFrame.sparse.from_spmatrix(vectores0,columns = vocabulario0)
df1 = pd.DataFrame.sparse.from_spmatrix(vectores1,columns = vocabulario1)



**Se agrega al dataframe de cada clase con el valor de label del que fueron seleccionados**

In [22]:
df0['clase'] = 0
df1['clase'] = 1

**Se combinan los dataframes df0 y df1 a partir de la columna clase**

In [23]:
dff = pd.merge(df0,df1, on = 'clase', how = 'outer')
print(dff)

   acoso_x  agar  arab  arar  bastar  bus  cadete  callate_x  camello  \
0      1.0   1.0   2.0   1.0     1.0  1.0     1.0        1.0      1.0   
1      NaN   NaN   NaN   NaN     NaN  NaN     NaN        NaN      NaN   

   categoria  ...  tiro  tres  turista  user  verdad  vida  viola  vía_y  \
0        1.0  ...   NaN   NaN      NaN   NaN     NaN   NaN    NaN    NaN   
1        NaN  ...   1.0   2.0      1.0   1.0     1.0   1.0    1.0    1.0   

   árabe_y   él  
0      NaN  NaN  
1      1.0  2.0  

[2 rows x 184 columns]


**Ya que la opción "outer" de la linea anterior nos ayuda a combinar los dataframes sin perder datos, está nos sobrescribe datos inexistentes con "NaN"
por lo tanto es mejor rellenarlos con ceros**

In [24]:
dff = dff.fillna(0)

**Mediante .pop quitamos la columna clase de dff y la guardamos en una variable auxiliar llamada dfff y mediante .insert la insertamos en la ultima posicion del dataframe** 

In [27]:
dfff = dff.pop('clase')
dff.insert(183, 'clase', dfff)

#### Finalmente se imprime el BoW final

In [34]:
dff.head()

,acoso_x,agar,arab,arar,bastar,bus,cadete,callate_x,camello,categoria,...,tres,turista,user,verdad,vida,viola,vía_y,árabe_y,él,clase
0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1
